In [ ]:
import pandas as pd

In [ ]:
#明智は人名、大湫はツイートなし、日吉は京都にも同じ地名あり、山岡は人名ラーメン屋、釜戸はご飯、明世はツイートなし、土岐は人名、陶は人名器
keyword_list = [#'sumou',
                'ena', 'nakatsugawa', 'mizunami', 'tajimi',
                #'iwamura', 'kushihara', 'ueyahagi', 'daishu', 'inetsu', # 分析可能
                #'meiyo', 'hiyoshi', 'kamako', 'tou', 'yamaoka', 'toki', 'akechi', #分析困難
                #'"obaachanshi"', '"saiensuwa-rudo"', '"kinaata" OR "kinaata"',
]
keyword = keyword_list[2]
period1 = '2018-07-18'
file1 = "tweets_in_a_week/keyword/"+keyword+period1+".csv" #edit!!!!!!!!!
period2 = '2018-07-25'
file2 = "tweets_in_a_week/keyword/"+keyword+period2+".csv" #edit!!!!!!!!!

In [ ]:
data1 = pd.read_csv(open(file1, 'rU'), encoding='utf-8')
data2 = pd.read_csv(open(file2, 'rU'), encoding='utf-8')

In [ ]:
datas = pd.concat([data1, data2], axis=0)
datas = datas.drop('Unnamed: 0', axis=1)
datas = datas.reset_index(drop=True)

# botとか不適切やツイートを除く

In [ ]:
datas

In [ ]:
improper_col = []
for col in range(datas.shape[0]):
    if type(datas.text[col]) != str:
        print('-'*40)
        print(col)
        print(type(datas.name[col]), type(datas.profile[col]), type(datas.text[col]),)
        print(datas.iloc[col])
        improper_col.append(col)
datas = datas.drop(improper_col, axis=0)
datas = datas.reset_index(drop=True)

In [ ]:
datas['n_following'] = datas['n_following'].astype('int')
datas['n_followed'] = datas['n_followed'].astype('int')
datas['n_tweets'] = datas['n_tweets'].astype('int')
datas['n_favorited'] = datas['n_favorited'].astype('int')

In [ ]:
improper_names  = ['bot', 'Bot', 'BOT', 'ぼっと', '情報', '案内', '相互', '出会', 'セフレ', 'エッチ', '法人',
                   'News', 'NEWS', 'news', 'ニュース', '新聞', '報道', 'NHK',
#                   '宮部雪宗', '多治見ネオ', 'キュアフランベ', 'SEATURTLES', '見習い魔法使い', '恵那豆', 'ゼイラム',
#                   'フィギュア', '空き家オールバンク', 'つけめん 恵那く', 'すぎたけ', '東城咲耶子', #for ena
#                   '石田麦', '創作さん', '恵那(癒し力高め中)', 'どう美じゅ合同誌', 'よつばと', #for ena
#                   '多治見でボイトレ', 'Macelle Yamamoto', '中野恵那', '王禅寺',
                   #'相撲 バズウォール', '逢華', '大西啓太', #for sumou
                  ] 
improper_profiles  = ['improper_words', 'bot', 'Bot', 'BOT', 'ぼっと',
#                      'えなfam', 'ｶﾞﾁだん', 'ガチだん', 'Καρδιά μου', '月がきれい', '成人済み腐女子', 'おりえなコンビ', #for ena
#                      'リトルアーモリー', 'アクティブなインドア派声優', 'ロマニとドレイク', '弱ペダ・東巻', #for ena
#                      'アイコンが中津川', 'チーム多治見',
                     ]
improper_texts  = ['improper_words', "I'm at", '相互', 'imacoconow',
#                   "稲津けんご", #for inatsu
#                   '斉藤恵那', '恵那さん', '恵那ちゃん', 'リトルアーモリー', 'よつばと', '小宮恵那', #for ena
#                   '中津川ひろさと', '塩尻', '松本', '恵那署', '斎藤恵那', 'ゆるキャン', '中野恵那',
                   #'火ノ丸', '大喜利', '手押し相撲' , '一人相撲' , '独り相撲', 'ローション', '名言集', '紙相撲', #for sumou
                   #'火の丸', 'カーネーション', '他人の褌で相撲を取る', '菊とギロチン', '伝令', '尻相撲', #for sumou
                   #'格闘技、プロ野球、相撲好き', '女相撲一座とアナキスト', '氷結相撲', #for sumou
                  ]

for col in range(datas.shape[0]):
    if type(datas['name'][col]) == str:
        for word in improper_names:
            if word in datas['name'][col]:
                datas['profile'][col] = 'improper_words'
                break
    if type(datas['profile'][col]) == str:
        for word in improper_profiles:
            if word in datas['profile'][col]:
                datas['text'][col] = 'improper_words'
                break
    for word in improper_texts:
        if word in datas['text'][col]:
            datas = datas.drop(col, axis=0)
            break

datas = datas.reset_index(drop=True)

# EDA

In [ ]:
n_keyword_tweets = datas['id'].value_counts()
n_keyword_tweets_arr = []

for col in range(datas.shape[0]):
    n_keyword_tweets_arr.append(n_keyword_tweets[datas['id'][col]])
datas['n_keyword_tweets'] = n_keyword_tweets_arr

del n_keyword_tweets, n_keyword_tweets_arr

In [ ]:
datas_unique = datas.drop_duplicates(subset='id')
datas_unique = datas_unique.reset_index(drop=True)

# キーワードを多くツイートした人

In [ ]:
datas_unique.sort_values(by='n_followed', ascending=False)

In [ ]:
influencer = datas_unique.sort_values(by='n_followed', ascending=False)[:100]
influencer.to_csv("tweets_in_a_week/keyword/"+keyword+period1+'_'+period2+"follower.csv", index=False)

In [ ]:
datas_unique.sort_values(by='n_favorited', ascending=False)

In [ ]:
influencer = datas_unique.sort_values(by='n_favorited', ascending=False)[:100]
influencer.to_csv("tweets_in_a_week/keyword/"+keyword+period1+'_'+period2+"favorited.csv", index=False)